In [492]:
import numpy as np
from sklearn.preprocessing import scale
import pandas as pd
from pandas import DataFrame, Series
import pyodbc
import plotly.plotly as py
import cufflinks as cf

In [493]:
array = []
city = []
uber = []
with open(r"C:\Users\Student\Documents\Uber_Project\data\CityStats.csv", 'r') as citystats:
    next(citystats)
    for line in citystats:
        array.append(line.replace("'",'').replace("\n",'').split(','))
        city.append(line.replace("'",'').replace("\n",'').split(',')[0])
        uber.append(line.replace("'",'').replace("\n",'').split(',')[-1])
    

In [494]:
city_np_array = np.array(array)

In [495]:
city_np_array_data= city_np_array.copy()
city_np_array_target= city_np_array.copy()

In [496]:
city_data = city_np_array_data[:,1:-1]

In [497]:
from sklearn.decomposition import RandomizedPCA
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6, random_state=0).fit(city_np_array_data[:,1:-1])
kmeans.labels_

array([5, 3, 5, 3, 0, 5, 2, 5, 5, 0, 0, 3, 4, 5, 3, 0, 5, 0, 3, 5, 5, 5, 0,
       3, 4, 5, 0, 5, 5, 4, 0, 4, 1, 5, 2, 5, 0, 5, 4, 5, 5, 5, 4, 3, 5, 5,
       0, 0, 3, 4, 3, 0, 0, 0, 5, 0, 5, 5, 5, 5, 5, 3, 5])

In [498]:
u = RandomizedPCA(n_components=2)
data = u.fit_transform(city_data)

C:\Users\Student\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning:

Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.



In [499]:
df = pd.DataFrame(city)

In [500]:
df['labels'] = kmeans.labels_.tolist()
df['uber'] = uber

In [501]:
df.columns=['City', 'Labels','Uber']
g5 = df[df.Labels==5]
g4 = df[df.Labels==4]
g3 = df[df.Labels==3]
g2 = df[df.Labels==2]
g1 = df[df.Labels==1]
g0 = df[df.Labels==0]

In [502]:
c = pyodbc.connect('DSN=sqluberconn;UID=tabi;PWD=password')

In [503]:
unemploy = pd.read_sql_query(sql = r'select [UALAD], [2011], [2012], [2013], [2014], [2015], [2016]from unemployment.[Rates_perc]', con = c)

In [504]:
g5 = g5.set_index(['City'])
g4 = g4.set_index(['City'])
g3 = g3.set_index(['City'])
g2 = g2.set_index(['City'])
g1 = g1.set_index(['City'])
g0 = g0.set_index(['City'])

In [505]:
unemploy = unemploy.set_index(['UALAD'])

In [506]:
g5 = pd.concat([g5, unemploy], axis = 1, join = 'inner')

In [507]:
g5_nuber = g5[g5.Uber=='0']

In [508]:
del g5_nuber['Labels']
del g5_nuber['Uber']

In [509]:
g5_uber = g5[g5.Uber=='1']
del g5_uber['Labels']
del g5_uber['Uber']

In [510]:
g3 = pd.concat([g3, unemploy], axis = 1, join = 'inner')

In [511]:
g3_nuber = g3[g3.Uber=='0']
del g3_nuber['Labels']
del g3_nuber['Uber']

In [512]:
g3_uber = g3[g3.Uber=='1']
del g3_uber['Labels']
del g3_uber['Uber']

In [513]:
g0 = pd.concat([g0, unemploy], axis = 1, join = 'inner')

In [514]:
g0_nuber = g0[g0.Uber=='0']
del g0_nuber['Labels']
del g0_nuber['Uber']

In [515]:
g0_uber = g0[g0.Uber=='1']
del g0_uber['Labels']
del g0_uber['Uber']

In [516]:
us = pd.read_csv(r'C:\Users\Student\Documents\Uber_Project\data\uberstartdates.csv')

In [517]:
us = us.set_index(['City'])

In [518]:
g5_uber = pd.concat([us, g5_uber], axis = 1, join = 'inner')

In [519]:
g5_uber14 = g5_uber[g5_uber.UberStart == 2014]

In [520]:
g5_uber15 = g5_uber[g5_uber.UberStart == 2015]

In [521]:
del g5_uber14['UberStart']
del g5_uber14['2011']
del g5_uber14['2012']

In [524]:
del g5_uber15['UberStart']
del g5_uber15['2011']
del g5_uber15['2012']

In [525]:
g5_uber14 = g5_uber14.apply(pd.to_numeric).mean()

In [526]:
g5_uber15 = g5_uber15.apply(pd.to_numeric).mean()

In [527]:
g5_nuber = g5_nuber.apply(pd.to_numeric).mean()

In [528]:
g5_ue = pd.concat([g5_uber14, g5_uber15, g5_nuber], axis = 1, join = 'inner')

In [529]:
g5_ue.columns = ['Uber 2014', 'Uber 2015', 'No Uber']

In [530]:
g5_layout = {'title': 'Group 5 Cities', 'xaxis':{'title': 'Year'}, 'yaxis':{'title': 'Unemployment Rate (%)'}}

In [531]:
g5_ue.iplot(kind = 'scatter', layout = g5_layout)

In [533]:
g3_uber = pd.concat([us, g3_uber], axis = 1, join = 'inner')

In [534]:
g0_uber = pd.concat([us, g0_uber], axis = 1, join = 'inner')

In [535]:
g0_uber14 = g0_uber[g0_uber.UberStart == 2014]

In [536]:
g0_uber15 = g0_uber[g0_uber.UberStart == 2015]

In [537]:
del g0_uber14['UberStart']
del g0_uber14['2011']
del g0_uber14['2012']

In [538]:
del g0_uber15['UberStart']
del g0_uber15['2011']
del g0_uber15['2012']

In [539]:
g0_uber14 = g0_uber14.apply(pd.to_numeric).mean()

In [540]:
g0_uber15 = g0_uber15.apply(pd.to_numeric).mean()

In [541]:
g0_nuber = g0_nuber.apply(pd.to_numeric).mean()

In [542]:
g0_ue = pd.concat([g0_uber14, g0_uber15, g0_nuber], axis = 1, join = 'inner')

In [543]:
g0_ue.columns = ['Uber 2014', 'Uber 2015', 'No Uber']

In [544]:
g0_layout = {'title': 'Group 0 Cities', 'xaxis':{'title': 'Year'}, 'yaxis':{'title': 'Unemployment Rate (%)'}}

In [545]:
g0_ue.iplot(kind = 'scatter', layout = g0_layout)